In [1]:
from __future__ import nested_scopes
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
display(HTML('<style>.CodeMirror{font-family: "Courier New";font-size: 12pt;}</style>'))

In [3]:
import findspark 
import sys
import os
import re
from datetime import date
from datetime import datetime

from pathlib import Path
home = "/mnt/workspace"


In [4]:
spark_home=f"{home}/xgb1.3/spark-3.0.1-bin-hadoop2.7"
java_home=f"{home}/jdk1.8.0_212"

os.environ["JAVA_HOME"]=java_home
findspark.init(spark_home)

!mkdir {home}/spark_events

spark_local_ramdisk=False

mkdir: cannot create directory ‘/mnt/workspace/spark_events’: File exists


In [5]:
import sys
sys.path.append(f'{home}/xgb1.3/xgboost4j-spark_3.0-1.3.0-0.1.0.jar')

In [6]:
# xgb1.3
!rm -rf {spark_home}/jars/rapids*
!cp {home}/xgb1.3/rapids-4-spark_2.12-0.3.0.jar {spark_home}/jars/

!rm -rf {spark_home}/jars/cudf*
!cp {home}/xgb1.3/cudf-0.17-cuda11.jar {spark_home}/jars/

rapidsjar=f"{home}/xgb1.3/rapids-4-spark_2.12-0.2.0.jar"
xgbversion="1.3.0"

In [7]:


jars=!ls -d {home}/xgb1.3/*{xgbversion}*
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars ' + ','.join(jars) + ' pyspark-shell'
%env PYSPARK_SUBMIT_ARGS

'--jars /mnt/workspace/xgb1.3/xgboost4j-spark_3.0-1.3.0-0.1.0.jar,/mnt/workspace/xgb1.3/xgboost4j_3.0-1.3.0-0.1.0.jar pyspark-shell'

In [8]:
for i in range(0,8):
    !mkdir /mnt/disk{i}/spark_local

mkdir: cannot create directory ‘/mnt/disk0/spark_local’: File exists
mkdir: cannot create directory ‘/mnt/disk1/spark_local’: File exists
mkdir: cannot create directory ‘/mnt/disk2/spark_local’: File exists
mkdir: cannot create directory ‘/mnt/disk3/spark_local’: File exists
mkdir: cannot create directory ‘/mnt/disk4/spark_local’: File exists
mkdir: cannot create directory ‘/mnt/disk5/spark_local’: File exists
mkdir: cannot create directory ‘/mnt/disk6/spark_local’: File exists
mkdir: cannot create directory ‘/mnt/disk7/spark_local’: File exists


In [9]:
",".join([f"/mnt/disk{i}/spark_local" for i in range(0,8)])

'/mnt/disk0/spark_local,/mnt/disk1/spark_local,/mnt/disk2/spark_local,/mnt/disk3/spark_local,/mnt/disk4/spark_local,/mnt/disk5/spark_local,/mnt/disk6/spark_local,/mnt/disk7/spark_local'

In [10]:
with open(f"{home}/getGpusResources.sh",'w') as f:
    f.write(r'''#!/usr/bin/env bash
ADDRS=`nvidia-smi --query-gpu=index --format=csv,noheader | sed -e ':a' -e 'N' -e'$!ba' -e 's/\n/","/g'`
echo {\"name\": \"gpu\", \"addresses\":[\"$ADDRS\"]}''')

with open(f"{spark_home}/conf/spark-defaults.conf",'w') as f:
    f.write('spark.worker.resource.gpu.amount 1\n')
    f.write(f'spark.worker.resource.gpu.discoveryScript {home}/getGpusResources.sh\n')
    f.write(f'spark.executor.resource.gpu.discoveryScript {home}/getGpusResources.sh\n')
    f.write(f'spark.history.fs.logDirectory file://{home}/spark_events\n')
    if spark_local_ramdisk:
        f.write(f'spark.local.dir {home}/spark_local_ramdisk\n')    
    else:
        f.write('spark.local.dir ' + ",".join([f"/mnt/disk{i}/spark_local" for i in range(0,8)]))    

In [11]:
!chmod +x {home}/getGpusResources.sh

In [12]:
!{home}/getGpusResources.sh

{"name": "gpu", "addresses":["0","1","2","3","4","5","6","7"]}


In [13]:
from ml.dmlc.xgboost4j.scala.spark import XGBoostClassificationModel, XGBoostClassifier,XGBoostRegressor
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql import SparkSession
from pyspark.sql.types import FloatType, IntegerType, StructField, StructType
import time, timeit
import subprocess

In [14]:
import pyspark
import math
import os
from pyspark.sql import functions as F
import pandas

In [15]:
!{spark_home}/sbin/stop-master.sh
!{spark_home}/sbin/stop-slave.sh

stopping org.apache.spark.deploy.master.Master
stopping org.apache.spark.deploy.worker.Worker


In [16]:
!{spark_home}/sbin/start-master.sh

starting org.apache.spark.deploy.master.Master, logging to /mnt/workspace/xgb1.3/spark-3.0.1-bin-hadoop2.7/logs/spark-ubuntu-org.apache.spark.deploy.master.Master-1-ip-172-31-29-158.out


In [17]:
!{spark_home}/sbin/start-slave.sh spark://ip-172-31-29-158.us-west-2.compute.internal:7077 -c 8

starting org.apache.spark.deploy.worker.Worker, logging to /mnt/workspace/xgb1.3/spark-3.0.1-bin-hadoop2.7/logs/spark-ubuntu-org.apache.spark.deploy.worker.Worker-1-ip-172-31-29-158.out


In [18]:
for l in range(0,8):
    !sudo nvidia-smi -i $l -c EXCLUSIVE_PROCESS

Compute mode is already set to EXCLUSIVE_PROCESS for GPU 00000000:10:1C.0.
All done.
Compute mode is already set to EXCLUSIVE_PROCESS for GPU 00000000:10:1D.0.
All done.
Compute mode is already set to EXCLUSIVE_PROCESS for GPU 00000000:20:1C.0.
All done.
Compute mode is already set to EXCLUSIVE_PROCESS for GPU 00000000:20:1D.0.
All done.
Compute mode is already set to EXCLUSIVE_PROCESS for GPU 00000000:90:1C.0.
All done.
Compute mode is already set to EXCLUSIVE_PROCESS for GPU 00000000:90:1D.0.
All done.
Compute mode is already set to EXCLUSIVE_PROCESS for GPU 00000000:A0:1C.0.
All done.
Compute mode is already set to EXCLUSIVE_PROCESS for GPU 00000000:A0:1D.0.
All done.


In [19]:
import socket
localhost=socket.gethostname()
clients=[localhost]

In [33]:


basedif="aws_a100"

emon_events='''
    -q -c -experimental -t0.5 -l100000 -u
    -C (

    INST_RETIRED.ANY
    CPU_CLK_UNHALTED.REF_TSC
    CPU_CLK_UNHALTED.THREAD
    CYCLE_ACTIVITY.STALLS_L3_MISS
    CYCLE_ACTIVITY.CYCLES_MEM_ANY
    CYCLE_ACTIVITY.STALLS_TOTAL
    DTLB_LOAD_MISSES.WALK_ACTIVE
    UNC_CHA_TOR_OCCUPANCY.IA_MISS:filter1=0x40432
    UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x40432
    UNC_CHA_CLOCKTICKS
    UNC_M_CAS_COUNT.RD
    UNC_M_CAS_COUNT.WR
    UNC_IIO_DATA_REQ_OF_CPU.MEM_WRITE.PART0
    UNC_IIO_DATA_REQ_OF_CPU.MEM_READ.PART0
    UNC_IIO_DATA_REQ_BY_CPU.MEM_WRITE.PART0
    UNC_IIO_DATA_REQ_BY_CPU.MEM_READ.PART0


    INST_RETIRED.ANY
    CPU_CLK_UNHALTED.REF_TSC
    CPU_CLK_UNHALTED.THREAD
    OFFCORE_RESPONSE:request=ALL_READS:response=L3_MISS.ANY_SNOOP:ocr_msr_val=0x3fB80007f7
    OFFCORE_RESPONSE:request=ALL_READS:response=L3_MISS.ANY_SNOOP:ocr_msr_val=0x3f840007f7
    MEM_LOAD_RETIRED.L3_MISS
    UOPS_ISSUED.STALL_CYCLES
    UNC_CHA_TOR_OCCUPANCY.IA_MISS:filter1=0x40431
    UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x40431
    UNC_CHA_CLOCKTICKS
    )
'''

def killsar(clients):
    for l in clients:
        out=!ssh $l "ps aux | grep -w sar | grep -v grep | tr -s ' ' | cut -d' ' -f2"
        for x in out:
            !ssh $l "kill $x > /dev/null 2>&1"
    for l in clients:
        out=!ssh $l "ps aux | grep -w pidstat | grep -v grep | tr -s ' ' | cut -d' ' -f2"
        for x in out:
            !ssh $l "kill $x > /dev/null 2>&1"
    for l in clients:
        out=!ssh $l "ps aux | grep -w perf | grep -v grep | tr -s ' ' | cut -d' ' -f2"
        for x in out:
            !ssh root@$l "kill $x > /dev/null 2>&1"
    for l in clients:
        out=!ssh $l "ps aux | grep -w nvidia-smi | grep -v grep | tr -s ' ' | cut -d' ' -f2"
        for x in out:
            !ssh $l "kill $x > /dev/null 2>&1"
    for l in clients:
        !ssh $l "source ~/sep_install/sep_vars.sh>/dev/null 2>&1; emon -stop > /dev/null 2>&1"
            
def startmonitor(clients,appid,**kwargs):
    local_profile_dir=home+"/profile/"
    prof=local_profile_dir+appid+"/"
    !mkdir -p $prof
    !echo "" > /tmp/training_time
    
    for l in clients:
        !ssh $l date
    
    killsar(clients)
    
    if kwargs.get("collect_emon",False):
        with open(home+"/emon.list",'w+') as f:
            f.write(emon_events)
        for l in clients:
            !scp {home}/emon.list {l}:{home}/emon.list  > /dev/null 2>&1
    
    perfsyscalls=kwargs.get("collect_perf_syscall",None)
    
    for l in clients:
        !mkdir -p $prof/$l/
        !ssh $l mkdir -p $prof/$l/
        !ssh $l "sar -o {prof}/{l}/sar.bin -r -u -d -n DEV 1 >/dev/null 2>&1 &"
        if kwargs.get("collect_pid",False):
            !ssh $l "{java_home}/bin/jps | grep CoarseGrainedExecutorBackend | head -n 1 | cut -d' ' -f 1 | xargs  -I % pidstat -h -t -p % 1  > {prof}/{l}/pidstat.out  2>/dev/null &"
        !ssh root@{l} 'cat /proc/uptime  | cut -d" " -f 1 | xargs -I ^ date -d "- ^ seconds"  +%s.%N' > $prof/$l/uptime.txt
        if kwargs.get("collect_sched",False):
            !ssh root@{l} 'perf trace -e "sched:sched_switch" -C 8-15 -o {prof}/{l}/sched.txt -T -- sleep 10000 >/dev/null 2>/dev/null &'
        if kwargs.get("collect_emon",False):
            !ssh {l} "source ~/sep_install/sep_vars.sh>/dev/null 2>&1; emon -i {home}/emon.list -f {prof}/{l}/emon.rst >/dev/null 2>&1 & "
        if perfsyscalls is not None:
            !ssh root@{l} "perf stat -e 'syscalls:sys_exit_poll,syscalls:sys_exit_epoll_wait' -a -I 1000 -o {prof}/{l}/perfstat.txt  >/dev/null 2>&1 & "
        if kwargs.get("collect_nv_smi",False):
            !ssh {l} "nvidia-smi --query-gpu=timestamp,index,utilization.gpu,utilization.memory,memory.used, -lms 500  --format=csv -f  {prof}/{l}/gpu.txt >/dev/null 2>&1 & "
    return prof

def stopmonitor(clients, sc, appid,**kwargs):
    %cd ~
    
    local_profile_dir=home+"/profile/"
    prof=local_profile_dir+appid+"/"
    !mkdir -p $prof
    
    killsar(clients)
    
    for l in clients:
        !ssh $l "sar -f {prof}/{l}/sar.bin -r > {prof}/{l}/sar_mem.sar;sar -f {prof}/{l}/sar.bin -u > {prof}/{l}/sar_cpu.sar;sar -f {prof}/{l}/sar.bin -d > {prof}/{l}/sar_disk.sar;sar -f {prof}/{l}/sar.bin -n DEV > {prof}/{l}/sar_nic.sar;" 
        !ssh $l "grep -rI xgbtck --no-filename {spark_home}/work/{appid}/* | sed 's/^ //g'  > {prof}/{l}/xgbtck.txt"
        if l!= socket.gethostname():
            !scp -r $l:$prof/$l $prof/ > /dev/null 2>&1
        !ssh $l "{java_home}/bin/jps | grep CoarseGrainedExecutorBackend | head -n 2 | tail -n 1 | cut -d' ' -f 1  | xargs -I % ps -To tid p %" > $prof/$l/sched_threads.txt
        !ssh $l "source ~/sep_install/sep_vars.sh>/dev/null 2>&1; emon -v " > $prof/$l/emonv.txt
        !ssh $l "sar -V " > $prof/$l/sarv.txt
        !test -f $prof/$l/perfstat.txt && head -n 1 $prof/$l/perfstat.txt > $prof/$l/perfstarttime
    if sc is not None:
        sc.stop()
    
    xgbfiles=!find {home}/profile/{appid} | grep xgbtck
    !cat /tmp/training_time >> {xgbfiles[0]}
    
#    !hadoop fs -copyToLocal /tmp/sparkEventLog/$appid $prof/app.log
    !cp {home}/spark_events/{appid}  $prof/app.log
#    fs.mkdirs(f"/{basedif}/")
#    v=[os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser(local_profile_dir+appid)) for f in fn]
#    for f in v:
#        paths=os.path.split(f)
#        fs.mkdirs("/"+basedif+"/"+paths[0][len(local_profile_dir):])
#        fs.copy_from_local(f,"/"+basedif+"/"+paths[0][len(local_profile_dir):]+"/"+paths[1],overwrite=True)
#    !HADOOP_USER_NAME=yuzhou hadoop fs -put  file://{prof} hdfs://10.1.2.125:8020/{basedif}
    print('http://10.0.0.34:18080/history/{:s}/jobs'.format(appid))
    print('http://sr525:1088/tracing_examples/trace_viewer.html#/tracing/test_data/{:s}.json'.format(appid))
    print(appid)

In [21]:
socket.gethostname()

'ip-172-31-29-158'

In [50]:
def create_cntx():

    from pyspark import SparkConf, SparkContext
    from pyspark.sql import SQLContext
    conf = (SparkConf()
        .set("spark.default.parallelism", "{:d}".format(nodes*executors_per_node*cores_per_executor*task_per_core))
        .set('spark.executor.instances', '{:d}'.format(nodes*executors_per_node))
        .set('spark.sql.files.maxPartitionBytes', '{:d}'.format(5*1024*1024*1024))
        .set('spark.app.name', 'pyspark_final')
        .set('spark.rdd.compress', 'False')
        .set('spark.executor.cores','{:d}'.format(executorcores))
        .set('spark.sql.shuffle.partitions','{:d}'.format(nodes*executors_per_node*cores_per_executor*task_per_core))
        .set('spark.executor.memory', '{:d}m'.format(int(math.floor(nodes*total_size/(nodes*executors_per_node)))-1024-int(math.floor(cache_size*1024/(nodes*executors_per_node)))))
        .set('spark.task.cpus',executorcores)
        .set('spark.driver.memory','4g')
        .set('spark.sql.inMemoryColumnarStorage.compressed','False')
        .set('spark.sql.inMemoryColumnarStorage.batchSize','100000')

        .set('spark.memory.offHeap.enabled','True')
        .set('spark.memory.offHeap.size','{:d}m'.format(int(math.floor(cache_size*1024/(nodes*executors_per_node)))))
        .set('spark.executor.memoryOverhead','{:d}m'.format(int(math.floor(cache_size*1024/(nodes*executors_per_node)))+3000))
        .set('spark.sql.join.preferSortMergeJoin','False')

        .set('spark.locality.wait','0s')

        .set('spark.rapids.memory.pinnedPool.size',"2G")
        .set('spark.sql.files.maxPartitionBytes','512m')
        .set('spark.sql.shuffle.partitions',executors_per_node)
        .set('spark.plugins','com.nvidia.spark.SQLPlugin')
        .set('spark.executor.resource.gpu.amount',1)
        .set('spark.task.resource.gpu.amount',1)
        .set('spark.rapids.sql.concurrentGpuTasks',1)
        .set('spark.kryo.registrator','com.nvidia.spark.rapids.GpuKryoRegistrator')
        .set('spark.worker.resource.gpu.amount',1)
        .set('spark.worker.resource.gpu.discoveryScript',f'{home}/getGpusResources.sh')
        .set('spark.rapids.sql.incompatibleOps.enabled',True)
        .set('spark.rapids.sql.hasNans',False)
        .set('spark.rapids.memory.gpu.pooling.enabled',False)
        .set('spark.eventLog.enabled',True)
        .set('spark.eventLog.dir',f'file://{home}/spark_events')
           )
    sc = SparkContext(conf=conf,master=f"spark://{localhost}:7077",appName='hibench')
    sc.setLogLevel("INFO")
    sc.addPyFile(f'{home}/xgb1.3/xgboost4j-spark_3.0-1.3.0-0.1.0.jar')
    sc.addPyFile(f'{home}/xgb1.3/rapids-4-spark_2.12-0.3.0.jar')
    
    for client in clients:
        pids=!ssh $client "{java_home}/bin/jps | grep CoarseGrainedExecutorBackend | cut -d' ' -f1"
        print(client,":",len(pids)," ","\t".join(map(str,pids)))

    print(sc.applicationId)
    spark = SQLContext(sc)
    return sc,spark

In [23]:
def run_train(train_data, params):
    t1 = timeit.default_timer()
    print("{} : {}".format("Start time", t1))
    singlePrecisionHistogram=params['singlePrecisionHistogram']
    del params['singlePrecisionHistogram']
    features=params['featuresCol']
    label=params['labelCol']
    xgboost =  XGBoostClassifier(**params)
    xgboost._java_obj.setSinglePrecisionHistogram(singlePrecisionHistogram)
    xgboost.setLabelCol(label).setFeaturesCols(features)
    model = xgboost.fit(train_data)
    t2 = timeit.default_timer()
    train_time = t2 - t1
    display(HTML(('Completed training the model. Time(sec): <font size=6pt color=red>{:f}</font>'.format(train_time))))
    return model, train_time

In [24]:
def load_parquet(label,**kwargs):
    file=kwargs.get("file",f"{home}/data/HiBench606Mx50.dataframe.float.parquet.8x96")
    train_data = spark.read.parquet(file)
    #train_data = train_data.coalesce(executors_per_no)
    schema = train_data.columns
    features = [ x for x in schema if x != label ]
    return (label,features,train_data)

In [25]:
def get_training_time_from_driver():
    driverlog=!cat {home}/notebook/nohup.out
    param=pandas.DataFrame([re.split("->",l) for l in driverlog if " -> " in l])
    start=[l for l in driverlog if "train-error:" in l or "train-rmse:" in l or "train-logloss" in l]
    end=[l for l in driverlog if "DEBUG Recieve shutdown signal" in l]
    x=re.search("(\d\d\d\d-\d\d-\d\d \d\d:\d\d:\d\d,\d\d\d)",start[0])
    if not x:
        return
    s=datetime.strptime(x.group(1), '%Y-%m-%d %H:%M:%S,%f')
    x=re.search("(\d\d\d\d-\d\d-\d\d \d\d:\d\d:\d\d,\d\d\d)",end[0])
    if not x:
        return
    e=datetime.strptime(x.group(1), '%Y-%m-%d %H:%M:%S,%f')

    trainingtime=e.timestamp()-s.timestamp()
    display(HTML(('Training Time(sec): <font size=6pt color=red>{:f}</font>'.format(trainingtime))))
    display(param)
    with open("/tmp/training_time","w") as f:
        f.write("container_1613068813053_0009_01_000002/stdout:xgbtck begin 0 {:d}\n".format(int(s.timestamp()*1000)))
        f.write("container_1613068813053_0009_01_000002/stdout:xgbtck end 0 {:d}\n".format(int(e.timestamp()*1000)))
        f.close()

In [26]:
def get_training_time_from_driver_oldway():
    driverlog=!cat {home}/notebook/nohup.out
    param=pandas.DataFrame([re.split("->",l) for l in driverlog if " -> " in l])
    start=[l for l in driverlog if "tracker All of" in l]
    end=[l for l in driverlog if "DEBUG Recieve shutdown signal" in l]
    x=re.search("(\d\d\d\d-\d\d-\d\d \d\d:\d\d:\d\d,\d\d\d)",start[0])
    if not x:
        return
    s=datetime.strptime(x.group(1), '%Y-%m-%d %H:%M:%S,%f')
    x=re.search("(\d\d\d\d-\d\d-\d\d \d\d:\d\d:\d\d,\d\d\d)",end[0])
    if not x:
        return
    e=datetime.strptime(x.group(1), '%Y-%m-%d %H:%M:%S,%f')

    trainingtime=e.timestamp()-s.timestamp()
    display(HTML(('Training Time(sec): <font size=6pt color=red>{:f}</font>'.format(trainingtime))))
    display(param)
    with open("/tmp/training_time","w") as f:
        f.write("container_1613068813053_0009_01_000002/stdout:xgbtck begin 0 {:d}\n".format(int(s.timestamp()*1000)))
        f.write("container_1613068813053_0009_01_000002/stdout:xgbtck end 0 {:d}\n".format(int(e.timestamp()*1000)))
        f.close()

In [27]:
def run_train_regressor(train_data, params):
    t1 = timeit.default_timer()
    print("{} : {}".format("Start time", t1))
    singlePrecisionHistogram=params['singlePrecisionHistogram']
    del params['singlePrecisionHistogram']
    features=params['featuresCol']
    label=params['labelCol']
    xgboost =  XGBoostRegressor(**params)
    xgboost._java_obj.setSinglePrecisionHistogram(singlePrecisionHistogram)
    xgboost.setLabelCol(label).setFeaturesCols(features)
    model = xgboost.fit(train_data)
    t2 = timeit.default_timer()
    train_time = t2 - t1
    display(HTML(('Completed training the model. Time(sec): <font size=6pt color=red>{:f}</font>'.format(train_time))))
    return model, train_time

In [28]:
def dropcache(clients):
    for l in clients:
        !ssh root@$l "sync && echo 3 > /proc/sys/vm/drop_caches; free -h"
        

In [29]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


# 600M, 8GPU, default param

In [30]:
executors_per_node = 8
nodes=1
cores_per_executor=1
task_per_core=1

executorcores=1

cache_size=2
total_size=500000
print('executor per node: {:d}\nparallelism: {:d}\nmemory: {:d}m\noffheap:{:d}m'.format(executors_per_node,nodes*executors_per_node*cores_per_executor*task_per_core,int(math.floor(nodes*total_size/(nodes*executors_per_node)))-1024-int(math.floor(cache_size*1024/(nodes*executors_per_node))),int(math.floor(cache_size*1024/(nodes*executors_per_node)))))

label="label"


for r in range(0,1):
    params = {'labelCol': "label",  #"label"
              'numRound': 100,
              'treeMethod': 'gpu_hist',
              'growPolicy': 'depthwise',  
              'singlePrecisionHistogram':True,
              'numWorkers': executors_per_node*nodes,
              'nthread':cores_per_executor,
              'verbosity':2
    }
#    dropcache(clients)
    sc, spark=create_cntx()
    appid = sc.applicationId
    try:
        !echo "" > {home}/notebook/nohup.out
        
        startmonitor(clients, appid, collect_emon=False, collect_nv_smi=True)
        t1 = timeit.default_timer()
        label, features,df = load_parquet(label)
        params['labelCol']=label
        params['featuresCol']=features
        model, train_time  = run_train(df, params)
        t2 = timeit.default_timer()
    
        get_training_time_from_driver_oldway()
        get_training_time_from_driver()
        stopmonitor(clients, sc, appid)
        display(HTML(('End 2 end. Time(sec): <font size=6pt color=red>{:f}</font>'.format(t2-t1))))

    finally:
        killsar(clients)
        sc.stop()

executor per node: 8
parallelism: 8
memory: 61220m
offheap:256m
ip-172-31-29-158 : 8   46626	46627	46628	46629	46630	46631	46632	46633
app-20210217040836-0000
Wed Feb 17 04:08:39 UTC 2021
root@ip-172-31-29-158: Permission denied (publickey).
Start time : 3266.011272147


,0,1,2,3,4,5,6
0,21/02/17 04:08:38 INFO ResourceProfile: Defaul...,"name: memoryOverhead, amount: 3256, script: ,...","name: cores, amount: 1, script: , vendor: , m...","name: memory, amount: 61220, script: , vendor...","name: gpu, amount: 1, script: /mnt/workspace/...","name: cpus, amount: 1.0, gpu","name: gpu, amount: 1.0)"
1,alpha,0.0,None,None,None,None,None
2,min_child_weight,1.0,None,None,None,None,None
3,sample_type,uniform,None,None,None,None,None
4,base_score,0.5,None,None,None,None,None
5,rabit_timeout,-1,None,None,None,None,None
6,colsample_bylevel,1.0,None,None,None,None,None
7,grow_policy,depthwise,None,None,None,None,None
8,skip_drop,0.0,None,None,None,None,None
9,lambda_bias,0.0,None,None,None,None,None


/home/ubuntu
bash: emon: command not found


NameError: name 'fs' is not defined

In [31]:
executors_per_node = 8
nodes=1
cores_per_executor=1
task_per_core=1

executorcores=1

cache_size=2
total_size=500000
print('executor per node: {:d}\nparallelism: {:d}\nmemory: {:d}m\noffheap:{:d}m'.format(executors_per_node,nodes*executors_per_node*cores_per_executor*task_per_core,int(math.floor(nodes*total_size/(nodes*executors_per_node)))-1024-int(math.floor(cache_size*1024/(nodes*executors_per_node))),int(math.floor(cache_size*1024/(nodes*executors_per_node)))))

label="label"


for r in range(0,2):
    params = {'labelCol': "label",  #"label"
              'numRound': 100,
              'treeMethod': 'gpu_hist',
              'growPolicy': 'depthwise',  
              'singlePrecisionHistogram':True,
              'numWorkers': executors_per_node*nodes,
              'nthread':cores_per_executor,
              'verbosity':2
    }
#    dropcache(clients)
    sc, spark=create_cntx()
    appid = sc.applicationId
    try:
        !echo "" > {home}/notebook/nohup.out
        
        startmonitor(clients, appid, collect_emon=False, collect_nv_smi=True)
        t1 = timeit.default_timer()
        label, features,df = load_parquet(label)
        params['labelCol']=label
        params['featuresCol']=features
        model, train_time  = run_train(df, params)
        t2 = timeit.default_timer()
    
        get_training_time_from_driver_oldway()
        get_training_time_from_driver()
        stopmonitor(clients, sc, appid)
        display(HTML(('End 2 end. Time(sec): <font size=6pt color=red>{:f}</font>'.format(t2-t1))))

    finally:
        killsar(clients)
        sc.stop()

executor per node: 8
parallelism: 8
memory: 61220m
offheap:256m
ip-172-31-29-158 : 8   51233	51249	51234	51238	51254	51241	51242	51245
app-20210217041539-0001
Wed Feb 17 04:15:41 UTC 2021
root@ip-172-31-29-158: Permission denied (publickey).
Start time : 3656.643657537


,0,1
0,alpha,0.0
1,min_child_weight,1.0
2,sample_type,uniform
3,base_score,0.5
4,rabit_timeout,-1
5,colsample_bylevel,1.0
6,grow_policy,depthwise
7,skip_drop,0.0
8,lambda_bias,0.0
9,silent,0


,0,1
0,alpha,0.0
1,min_child_weight,1.0
2,sample_type,uniform
3,base_score,0.5
4,rabit_timeout,-1
5,colsample_bylevel,1.0
6,grow_policy,depthwise
7,skip_drop,0.0
8,lambda_bias,0.0
9,silent,0


/home/ubuntu
bash: emon: command not found


NameError: name 'fs' is not defined

In [34]:
executors_per_node = 8
nodes=1
cores_per_executor=1
task_per_core=1

executorcores=1

cache_size=2
total_size=500000
print('executor per node: {:d}\nparallelism: {:d}\nmemory: {:d}m\noffheap:{:d}m'.format(executors_per_node,nodes*executors_per_node*cores_per_executor*task_per_core,int(math.floor(nodes*total_size/(nodes*executors_per_node)))-1024-int(math.floor(cache_size*1024/(nodes*executors_per_node))),int(math.floor(cache_size*1024/(nodes*executors_per_node)))))

label="label"


for r in range(0,1):
    params = {'labelCol': "label",  #"label"
              'numRound': 100,
              'treeMethod': 'gpu_hist',
              'growPolicy': 'depthwise',  
              'singlePrecisionHistogram':True,
              'numWorkers': executors_per_node*nodes,
              'nthread':cores_per_executor,
              'verbosity':2
    }
#    dropcache(clients)
    sc, spark=create_cntx()
    appid = sc.applicationId
    try:
        !echo "" > {home}/notebook/nohup.out
        
        startmonitor(clients, appid, collect_emon=False, collect_nv_smi=True)
        t1 = timeit.default_timer()
        label, features,df = load_parquet(label)
        params['labelCol']=label
        params['featuresCol']=features
        model, train_time  = run_train(df, params)
        t2 = timeit.default_timer()
    
        get_training_time_from_driver_oldway()
        get_training_time_from_driver()
        stopmonitor(clients, sc, appid)
        display(HTML(('End 2 end. Time(sec): <font size=6pt color=red>{:f}</font>'.format(t2-t1))))

    finally:
        killsar(clients)
        sc.stop()

executor per node: 8
parallelism: 8
memory: 61220m
offheap:256m
ip-172-31-29-158 : 8   56048	56033	56037	56040	56057	56059	56047	56031
app-20210217042341-0002
Wed Feb 17 04:23:42 UTC 2021
root@ip-172-31-29-158: Permission denied (publickey).
Start time : 4147.552017022


,0,1
0,alpha,0.0
1,min_child_weight,1.0
2,sample_type,uniform
3,base_score,0.5
4,rabit_timeout,-1
5,colsample_bylevel,1.0
6,grow_policy,depthwise
7,skip_drop,0.0
8,lambda_bias,0.0
9,silent,0


,0,1
0,alpha,0.0
1,min_child_weight,1.0
2,sample_type,uniform
3,base_score,0.5
4,rabit_timeout,-1
5,colsample_bylevel,1.0
6,grow_policy,depthwise
7,skip_drop,0.0
8,lambda_bias,0.0
9,silent,0


/home/ubuntu
bash: emon: command not found
http://10.0.0.34:18080/history/app-20210217042341-0002/jobs
http://sr525:1088/tracing_examples/trace_viewer.html#/tracing/test_data/app-20210217042341-0002.json
app-20210217042341-0002


In [59]:
executors_per_node = 8
nodes=1
cores_per_executor=1
task_per_core=1

executorcores=1

cache_size=2
total_size=50000
print('executor per node: {:d}\nparallelism: {:d}\nmemory: {:d}m\noffheap:{:d}m'.format(executors_per_node,nodes*executors_per_node*cores_per_executor*task_per_core,int(math.floor(nodes*total_size/(nodes*executors_per_node)))-1024-int(math.floor(cache_size*1024/(nodes*executors_per_node))),int(math.floor(cache_size*1024/(nodes*executors_per_node)))))
label="label"
sc, spark=create_cntx()


executor per node: 8
parallelism: 8
memory: 4970m
offheap:256m
ip-172-31-29-158 : 8   26932	26935	26936	26937	26925	26926	26942	26927
app-20210217060315-0018


In [60]:
df=spark.read.parquet(f"{home}/data/HiBench606Mx50.dataframe.float.parquet.8x96")

In [61]:
df=df.repartition(8)
df.write.parquet(f"{home}/data/HiBench606Mx50.dataframe.float.parquet.8")

In [62]:
sc.stop()

In [63]:
executors_per_node = 8
nodes=1
cores_per_executor=1
task_per_core=1

executorcores=1

cache_size=2
total_size=500000
print('executor per node: {:d}\nparallelism: {:d}\nmemory: {:d}m\noffheap:{:d}m'.format(executors_per_node,nodes*executors_per_node*cores_per_executor*task_per_core,int(math.floor(nodes*total_size/(nodes*executors_per_node)))-1024-int(math.floor(cache_size*1024/(nodes*executors_per_node))),int(math.floor(cache_size*1024/(nodes*executors_per_node)))))

label="label"


for r in range(0,1):
    params = {'labelCol': "label",  #"label"
              'numRound': 100,
              'treeMethod': 'gpu_hist',
              'growPolicy': 'depthwise',  
              'singlePrecisionHistogram':True,
              'numWorkers': executors_per_node*nodes,
              'nthread':cores_per_executor,
              'verbosity':2
    }
#    dropcache(clients)
    sc, spark=create_cntx()
    appid = sc.applicationId
    try:
        !echo "" > {home}/notebook/nohup.out
        
        startmonitor(clients, appid, collect_emon=False, collect_nv_smi=True)
        t1 = timeit.default_timer()
        label, features,df = load_parquet(label,file=f"{home}/data/HiBench606Mx50.dataframe.float.parquet.8")
        params['labelCol']=label
        params['featuresCol']=features
        model, train_time  = run_train(df, params)
        t2 = timeit.default_timer()
    
        get_training_time_from_driver_oldway()
        get_training_time_from_driver()
        stopmonitor(clients, sc, appid)
        display(HTML(('End 2 end. Time(sec): <font size=6pt color=red>{:f}</font>'.format(t2-t1))))

    finally:
        killsar(clients)
        sc.stop()

executor per node: 8
parallelism: 8
memory: 61220m
offheap:256m
ip-172-31-29-158 : 8   29450	29451	29458	29459	29460	29461	29462	29465
app-20210217060929-0019
Wed Feb 17 06:09:31 UTC 2021
root@ip-172-31-29-158: Permission denied (publickey).
Start time : 10497.023053592


,0,1
0,alpha,0.0
1,min_child_weight,1.0
2,sample_type,uniform
3,base_score,0.5
4,rabit_timeout,-1
5,colsample_bylevel,1.0
6,grow_policy,depthwise
7,skip_drop,0.0
8,lambda_bias,0.0
9,silent,0


,0,1
0,alpha,0.0
1,min_child_weight,1.0
2,sample_type,uniform
3,base_score,0.5
4,rabit_timeout,-1
5,colsample_bylevel,1.0
6,grow_policy,depthwise
7,skip_drop,0.0
8,lambda_bias,0.0
9,silent,0


/home/ubuntu
bash: emon: command not found
http://10.0.0.34:18080/history/app-20210217060929-0019/jobs
http://sr525:1088/tracing_examples/trace_viewer.html#/tracing/test_data/app-20210217060929-0019.json
app-20210217060929-0019


In [64]:
executors_per_node = 8
nodes=1
cores_per_executor=1
task_per_core=1

executorcores=1

cache_size=2
total_size=500000
print('executor per node: {:d}\nparallelism: {:d}\nmemory: {:d}m\noffheap:{:d}m'.format(executors_per_node,nodes*executors_per_node*cores_per_executor*task_per_core,int(math.floor(nodes*total_size/(nodes*executors_per_node)))-1024-int(math.floor(cache_size*1024/(nodes*executors_per_node))),int(math.floor(cache_size*1024/(nodes*executors_per_node)))))

label="label"


for r in range(0,1):
    params = {'labelCol': "label",  #"label"
              'numRound': 100,
              'treeMethod': 'gpu_hist',
              'growPolicy': 'depthwise',  
              'singlePrecisionHistogram':True,
              'numWorkers': executors_per_node*nodes,
              'nthread':cores_per_executor,
              'verbosity':2
    }
#    dropcache(clients)
    sc, spark=create_cntx()
    appid = sc.applicationId
    try:
        !echo "" > {home}/notebook/nohup.out
        
        startmonitor(clients, appid, collect_emon=False, collect_nv_smi=True)
        t1 = timeit.default_timer()
        label, features,df = load_parquet(label,file=f"{home}/data/HiBench606Mx50.dataframe.float.parquet.8")
        params['labelCol']=label
        params['featuresCol']=features
        model, train_time  = run_train(df, params)
        t2 = timeit.default_timer()
    
        get_training_time_from_driver_oldway()
        get_training_time_from_driver()
        stopmonitor(clients, sc, appid)
        display(HTML(('End 2 end. Time(sec): <font size=6pt color=red>{:f}</font>'.format(t2-t1))))

    finally:
        killsar(clients)
        sc.stop()

executor per node: 8
parallelism: 8
memory: 61220m
offheap:256m
ip-172-31-29-158 : 8   33865	33866	33867	33868	33871	33878	33886	33887
app-20210217061406-0020
Wed Feb 17 06:14:08 UTC 2021
root@ip-172-31-29-158: Permission denied (publickey).
Start time : 10774.043363836


,0,1
0,alpha,0.0
1,min_child_weight,1.0
2,sample_type,uniform
3,base_score,0.5
4,rabit_timeout,-1
5,colsample_bylevel,1.0
6,grow_policy,depthwise
7,skip_drop,0.0
8,lambda_bias,0.0
9,silent,0


,0,1
0,alpha,0.0
1,min_child_weight,1.0
2,sample_type,uniform
3,base_score,0.5
4,rabit_timeout,-1
5,colsample_bylevel,1.0
6,grow_policy,depthwise
7,skip_drop,0.0
8,lambda_bias,0.0
9,silent,0


/home/ubuntu
bash: emon: command not found
http://10.0.0.34:18080/history/app-20210217061406-0020/jobs
http://sr525:1088/tracing_examples/trace_viewer.html#/tracing/test_data/app-20210217061406-0020.json
app-20210217061406-0020


# 900M Row

In [35]:
executors_per_node = 8
nodes=1
cores_per_executor=1
task_per_core=1

executorcores=1

cache_size=2
total_size=50000
print('executor per node: {:d}\nparallelism: {:d}\nmemory: {:d}m\noffheap:{:d}m'.format(executors_per_node,nodes*executors_per_node*cores_per_executor*task_per_core,int(math.floor(nodes*total_size/(nodes*executors_per_node)))-1024-int(math.floor(cache_size*1024/(nodes*executors_per_node))),int(math.floor(cache_size*1024/(nodes*executors_per_node)))))

label="label"


for r in range(0,2):
    params = {'labelCol': "label",  #"label"
              'numRound': 100,
              'treeMethod': 'gpu_hist',
              'growPolicy': 'depthwise',  
              'singlePrecisionHistogram':True,
              'numWorkers': executors_per_node*nodes,
              'nthread':cores_per_executor,
              'verbosity':2
    }
#    dropcache(clients)
    sc, spark=create_cntx()
    appid = sc.applicationId
    try:
        !echo "" > {home}/notebook/nohup.out
        
        startmonitor(clients, appid, collect_emon=True, collect_nv_smi=True)
        t1 = timeit.default_timer()
        label, features,df = load_parquet(label,file="/mnt/workspace/data/HiBench900Mx50.dataframe.float.parquet.8x96")
        params['labelCol']=label
        params['featuresCol']=features
        model, train_time  = run_train(df, params)
        t2 = timeit.default_timer()
    
        get_training_time_from_driver_oldway()
        get_training_time_from_driver()
        stopmonitor(clients, sc, appid)
        display(HTML(('End 2 end. Time(sec): <font size=6pt color=red>{:f}</font>'.format(t2-t1))))
    finally:
        killsar(clients)
        sc.stop()

executor per node: 8
parallelism: 8
memory: 4970m
offheap:256m
ip-172-31-29-158 : 8   60513	60514	60516	60519	60520	60521	60507	60510
app-20210217042939-0003
Wed Feb 17 04:29:40 UTC 2021
root@ip-172-31-29-158: Permission denied (publickey).
Start time : 4510.618823186


,0,1
0,alpha,0.0
1,min_child_weight,1.0
2,sample_type,uniform
3,base_score,0.5
4,rabit_timeout,-1
5,colsample_bylevel,1.0
6,grow_policy,depthwise
7,skip_drop,0.0
8,lambda_bias,0.0
9,silent,0


,0,1
0,alpha,0.0
1,min_child_weight,1.0
2,sample_type,uniform
3,base_score,0.5
4,rabit_timeout,-1
5,colsample_bylevel,1.0
6,grow_policy,depthwise
7,skip_drop,0.0
8,lambda_bias,0.0
9,silent,0


/home/ubuntu
bash: emon: command not found
http://10.0.0.34:18080/history/app-20210217042939-0003/jobs
http://sr525:1088/tracing_examples/trace_viewer.html#/tracing/test_data/app-20210217042939-0003.json
app-20210217042939-0003


ip-172-31-29-158 : 8   65232	65233	65234	65235	65258	65229	65230	65231
app-20210217043550-0004
Wed Feb 17 04:35:51 UTC 2021
root@ip-172-31-29-158: Permission denied (publickey).
Start time : 4901.139305086


,0,1
0,alpha,0.0
1,min_child_weight,1.0
2,sample_type,uniform
3,base_score,0.5
4,rabit_timeout,-1
5,colsample_bylevel,1.0
6,grow_policy,depthwise
7,skip_drop,0.0
8,lambda_bias,0.0
9,silent,0


,0,1
0,alpha,0.0
1,min_child_weight,1.0
2,sample_type,uniform
3,base_score,0.5
4,rabit_timeout,-1
5,colsample_bylevel,1.0
6,grow_policy,depthwise
7,skip_drop,0.0
8,lambda_bias,0.0
9,silent,0


/home/ubuntu
bash: emon: command not found
http://10.0.0.34:18080/history/app-20210217043550-0004/jobs
http://sr525:1088/tracing_examples/trace_viewer.html#/tracing/test_data/app-20210217043550-0004.json
app-20210217043550-0004


In [69]:
executors_per_node = 8
nodes=1
cores_per_executor=1
task_per_core=1

executorcores=1

cache_size=2
total_size=50000
print('executor per node: {:d}\nparallelism: {:d}\nmemory: {:d}m\noffheap:{:d}m'.format(executors_per_node,nodes*executors_per_node*cores_per_executor*task_per_core,int(math.floor(nodes*total_size/(nodes*executors_per_node)))-1024-int(math.floor(cache_size*1024/(nodes*executors_per_node))),int(math.floor(cache_size*1024/(nodes*executors_per_node)))))
label="label"
sc, spark=create_cntx()


executor per node: 8
parallelism: 8
memory: 4970m
offheap:256m
ip-172-31-29-158 : 8   41056	41058	41040	41044	41046	41049	41051	41053
app-20210217062922-0022


In [70]:
df=spark.read.parquet(f"{home}/data/HiBench900Mx50.dataframe.float.parquet.8x96")

In [71]:
df=df.repartition(16)
df.write.parquet(f"{home}/data/HiBench900Mx50.dataframe.float.parquet.16")

In [72]:
sc.stop()

In [73]:
executors_per_node = 8
nodes=1
cores_per_executor=1
task_per_core=1

executorcores=1

cache_size=2
total_size=50000
print('executor per node: {:d}\nparallelism: {:d}\nmemory: {:d}m\noffheap:{:d}m'.format(executors_per_node,nodes*executors_per_node*cores_per_executor*task_per_core,int(math.floor(nodes*total_size/(nodes*executors_per_node)))-1024-int(math.floor(cache_size*1024/(nodes*executors_per_node))),int(math.floor(cache_size*1024/(nodes*executors_per_node)))))

label="label"


for r in range(0,1):
    params = {'labelCol': "label",  #"label"
              'numRound': 100,
              'treeMethod': 'gpu_hist',
              'growPolicy': 'depthwise',  
              'singlePrecisionHistogram':True,
              'numWorkers': executors_per_node*nodes,
              'nthread':cores_per_executor,
              'verbosity':2
    }
#    dropcache(clients)
    sc, spark=create_cntx()
    appid = sc.applicationId
    try:
        !echo "" > {home}/notebook/nohup.out
        
        startmonitor(clients, appid, collect_emon=True, collect_nv_smi=True)
        t1 = timeit.default_timer()
        label, features,df = load_parquet(label,file="/mnt/workspace/data/HiBench900Mx50.dataframe.float.parquet.16")
        params['labelCol']=label
        params['featuresCol']=features
        model, train_time  = run_train(df, params)
        t2 = timeit.default_timer()
    
        get_training_time_from_driver_oldway()
        get_training_time_from_driver()
        stopmonitor(clients, sc, appid)
        display(HTML(('End 2 end. Time(sec): <font size=6pt color=red>{:f}</font>'.format(t2-t1))))
    finally:
        killsar(clients)
        sc.stop()

executor per node: 8
parallelism: 8
memory: 4970m
offheap:256m
ip-172-31-29-158 : 8   44680	44650	44651	44652	44654	44656	44660	44662
app-20210217063436-0023
Wed Feb 17 06:34:38 UTC 2021
root@ip-172-31-29-158: Permission denied (publickey).
Start time : 12029.297037386


,0,1
0,alpha,0.0
1,min_child_weight,1.0
2,sample_type,uniform
3,base_score,0.5
4,rabit_timeout,-1
5,colsample_bylevel,1.0
6,grow_policy,depthwise
7,skip_drop,0.0
8,lambda_bias,0.0
9,silent,0


,0,1
0,alpha,0.0
1,min_child_weight,1.0
2,sample_type,uniform
3,base_score,0.5
4,rabit_timeout,-1
5,colsample_bylevel,1.0
6,grow_policy,depthwise
7,skip_drop,0.0
8,lambda_bias,0.0
9,silent,0


/home/ubuntu
bash: emon: command not found
http://10.0.0.34:18080/history/app-20210217063436-0023/jobs
http://sr525:1088/tracing_examples/trace_viewer.html#/tracing/test_data/app-20210217063436-0023.json
app-20210217063436-0023


# 1200M

## create data set

In [74]:
executors_per_node = 8
nodes=1
cores_per_executor=1
task_per_core=1

executorcores=1

cache_size=2
total_size=50000
print('executor per node: {:d}\nparallelism: {:d}\nmemory: {:d}m\noffheap:{:d}m'.format(executors_per_node,nodes*executors_per_node*cores_per_executor*task_per_core,int(math.floor(nodes*total_size/(nodes*executors_per_node)))-1024-int(math.floor(cache_size*1024/(nodes*executors_per_node))),int(math.floor(cache_size*1024/(nodes*executors_per_node)))))
label="label"
sc, spark=create_cntx()


executor per node: 8
parallelism: 8
memory: 4970m
offheap:256m
ip-172-31-29-158 : 8   50249	50250	50225	50226	50228	50229	50230	50233
app-20210217064034-0024


In [75]:
df=spark.read.parquet(f"{home}/data/HiBench606Mx50.dataframe.float.parquet.8")

In [76]:
df=df.union(df).repartition(16)
df.write.parquet(f"{home}/data/HiBench1200Mx50.dataframe.float.parquet.16")

In [77]:
sc.stop()

## 1200M Row

In [40]:
executors_per_node = 8
nodes=1
cores_per_executor=1
task_per_core=1

executorcores=1

cache_size=2
total_size=50000
print('executor per node: {:d}\nparallelism: {:d}\nmemory: {:d}m\noffheap:{:d}m'.format(executors_per_node,nodes*executors_per_node*cores_per_executor*task_per_core,int(math.floor(nodes*total_size/(nodes*executors_per_node)))-1024-int(math.floor(cache_size*1024/(nodes*executors_per_node))),int(math.floor(cache_size*1024/(nodes*executors_per_node)))))

label="label"


for r in range(0,2):
    params = {'labelCol': "label",  #"label"
              'numRound': 100,
              'treeMethod': 'gpu_hist',
              'growPolicy': 'depthwise',  
              'singlePrecisionHistogram':True,
              'numWorkers': executors_per_node*nodes,
              'nthread':cores_per_executor,
              'verbosity':2
    }
#    dropcache(clients)
    sc, spark=create_cntx()
    appid = sc.applicationId
    try:
        !echo "" > {home}/notebook/nohup.out
        
        startmonitor(clients, appid, collect_emon=True, collect_nv_smi=True)
        t1 = timeit.default_timer()
        label, features,df = load_parquet(label,file=f"{home}/data/HiBench1200Mx50.dataframe.float.parquet.2x8x96")
        params['labelCol']=label
        params['featuresCol']=features
        model, train_time  = run_train(df, params)
        t2 = timeit.default_timer()
    
        get_training_time_from_driver_oldway()
        get_training_time_from_driver()
        stopmonitor(clients, sc, appid)
        display(HTML(('End 2 end. Time(sec): <font size=6pt color=red>{:f}</font>'.format(t2-t1))))
    finally:
        killsar(clients)
        sc.stop()

executor per node: 8
parallelism: 8
memory: 4970m
offheap:256m
ip-172-31-29-158 : 8   74272	74259	74260	74267	74266	74253	74269	74255
app-20210217045108-0006
Wed Feb 17 04:51:10 UTC 2021
root@ip-172-31-29-158: Permission denied (publickey).
Start time : 5807.320033554


,0,1
0,alpha,0.0
1,min_child_weight,1.0
2,sample_type,uniform
3,base_score,0.5
4,rabit_timeout,-1
5,colsample_bylevel,1.0
6,grow_policy,depthwise
7,skip_drop,0.0
8,lambda_bias,0.0
9,silent,0


,0,1
0,alpha,0.0
1,min_child_weight,1.0
2,sample_type,uniform
3,base_score,0.5
4,rabit_timeout,-1
5,colsample_bylevel,1.0
6,grow_policy,depthwise
7,skip_drop,0.0
8,lambda_bias,0.0
9,silent,0


/home/ubuntu
bash: emon: command not found
http://10.0.0.34:18080/history/app-20210217045108-0006/jobs
http://sr525:1088/tracing_examples/trace_viewer.html#/tracing/test_data/app-20210217045108-0006.json
app-20210217045108-0006


ip-172-31-29-158 : 8   78945	78944	78947	78962	78946	78953	78955	78956
app-20210217045937-0007
Wed Feb 17 04:59:38 UTC 2021
root@ip-172-31-29-158: Permission denied (publickey).
Start time : 6295.368753206


,0,1
0,alpha,0.0
1,min_child_weight,1.0
2,sample_type,uniform
3,base_score,0.5
4,rabit_timeout,-1
5,colsample_bylevel,1.0
6,grow_policy,depthwise
7,skip_drop,0.0
8,lambda_bias,0.0
9,silent,0


,0,1
0,alpha,0.0
1,min_child_weight,1.0
2,sample_type,uniform
3,base_score,0.5
4,rabit_timeout,-1
5,colsample_bylevel,1.0
6,grow_policy,depthwise
7,skip_drop,0.0
8,lambda_bias,0.0
9,silent,0


/home/ubuntu
bash: emon: command not found
http://10.0.0.34:18080/history/app-20210217045937-0007/jobs
http://sr525:1088/tracing_examples/trace_viewer.html#/tracing/test_data/app-20210217045937-0007.json
app-20210217045937-0007


In [78]:
executors_per_node = 8
nodes=1
cores_per_executor=1
task_per_core=1

executorcores=1

cache_size=2
total_size=50000
print('executor per node: {:d}\nparallelism: {:d}\nmemory: {:d}m\noffheap:{:d}m'.format(executors_per_node,nodes*executors_per_node*cores_per_executor*task_per_core,int(math.floor(nodes*total_size/(nodes*executors_per_node)))-1024-int(math.floor(cache_size*1024/(nodes*executors_per_node))),int(math.floor(cache_size*1024/(nodes*executors_per_node)))))

label="label"


for r in range(0,1):
    params = {'labelCol': "label",  #"label"
              'numRound': 100,
              'treeMethod': 'gpu_hist',
              'growPolicy': 'depthwise',  
              'singlePrecisionHistogram':True,
              'numWorkers': executors_per_node*nodes,
              'nthread':cores_per_executor,
              'verbosity':2
    }
#    dropcache(clients)
    sc, spark=create_cntx()
    appid = sc.applicationId
    try:
        !echo "" > {home}/notebook/nohup.out
        
        startmonitor(clients, appid, collect_emon=True, collect_nv_smi=True)
        t1 = timeit.default_timer()
        label, features,df = load_parquet(label,file=f"{home}/data/HiBench1200Mx50.dataframe.float.parquet.16")
        params['labelCol']=label
        params['featuresCol']=features
        model, train_time  = run_train(df, params)
        t2 = timeit.default_timer()
    
        get_training_time_from_driver_oldway()
        get_training_time_from_driver()
        stopmonitor(clients, sc, appid)
        display(HTML(('End 2 end. Time(sec): <font size=6pt color=red>{:f}</font>'.format(t2-t1))))
    finally:
        killsar(clients)
        sc.stop()

executor per node: 8
parallelism: 8
memory: 4970m
offheap:256m
ip-172-31-29-158 : 8   53225	52875	52879	52853	52854	53142	52855	52856
app-20210217065631-0025
Wed Feb 17 06:56:33 UTC 2021
root@ip-172-31-29-158: Permission denied (publickey).
Start time : 13349.500396686


,0,1
0,alpha,0.0
1,min_child_weight,1.0
2,sample_type,uniform
3,base_score,0.5
4,rabit_timeout,-1
5,colsample_bylevel,1.0
6,grow_policy,depthwise
7,skip_drop,0.0
8,lambda_bias,0.0
9,silent,0


,0,1
0,alpha,0.0
1,min_child_weight,1.0
2,sample_type,uniform
3,base_score,0.5
4,rabit_timeout,-1
5,colsample_bylevel,1.0
6,grow_policy,depthwise
7,skip_drop,0.0
8,lambda_bias,0.0
9,silent,0


/home/ubuntu
bash: emon: command not found
http://10.0.0.34:18080/history/app-20210217065631-0025/jobs
http://sr525:1088/tracing_examples/trace_viewer.html#/tracing/test_data/app-20210217065631-0025.json
app-20210217065631-0025


In [79]:
executors_per_node = 8
nodes=1
cores_per_executor=1
task_per_core=1

executorcores=1

cache_size=2
total_size=50000
print('executor per node: {:d}\nparallelism: {:d}\nmemory: {:d}m\noffheap:{:d}m'.format(executors_per_node,nodes*executors_per_node*cores_per_executor*task_per_core,int(math.floor(nodes*total_size/(nodes*executors_per_node)))-1024-int(math.floor(cache_size*1024/(nodes*executors_per_node))),int(math.floor(cache_size*1024/(nodes*executors_per_node)))))

label="label"


for r in range(0,1):
    params = {'labelCol': "label",  #"label"
              'numRound': 100,
              'treeMethod': 'gpu_hist',
              'growPolicy': 'depthwise',  
              'singlePrecisionHistogram':True,
              'numWorkers': executors_per_node*nodes,
              'nthread':cores_per_executor,
              'verbosity':2
    }
#    dropcache(clients)
    sc, spark=create_cntx()
    appid = sc.applicationId
    try:
        !echo "" > {home}/notebook/nohup.out
        
        startmonitor(clients, appid, collect_emon=True, collect_nv_smi=True)
        t1 = timeit.default_timer()
        label, features,df = load_parquet(label,file=f"{home}/data/HiBench1200Mx50.dataframe.float.parquet.16")
        params['labelCol']=label
        params['featuresCol']=features
        model, train_time  = run_train(df, params)
        t2 = timeit.default_timer()
    
        get_training_time_from_driver_oldway()
        get_training_time_from_driver()
        stopmonitor(clients, sc, appid)
        display(HTML(('End 2 end. Time(sec): <font size=6pt color=red>{:f}</font>'.format(t2-t1))))
    finally:
        killsar(clients)
        sc.stop()

executor per node: 8
parallelism: 8
memory: 4970m
offheap:256m
ip-172-31-29-158 : 8   58349	58350	58351	58352	58331	58332	58333	58334
app-20210217070653-0026
Wed Feb 17 07:06:55 UTC 2021
root@ip-172-31-29-158: Permission denied (publickey).
Start time : 13947.116349357


,0,1
0,alpha,0.0
1,min_child_weight,1.0
2,sample_type,uniform
3,base_score,0.5
4,rabit_timeout,-1
5,colsample_bylevel,1.0
6,grow_policy,depthwise
7,skip_drop,0.0
8,lambda_bias,0.0
9,silent,0


,0,1
0,alpha,0.0
1,min_child_weight,1.0
2,sample_type,uniform
3,base_score,0.5
4,rabit_timeout,-1
5,colsample_bylevel,1.0
6,grow_policy,depthwise
7,skip_drop,0.0
8,lambda_bias,0.0
9,silent,0


/home/ubuntu
bash: emon: command not found
http://10.0.0.34:18080/history/app-20210217070653-0026/jobs
http://sr525:1088/tracing_examples/trace_viewer.html#/tracing/test_data/app-20210217070653-0026.json
app-20210217070653-0026


# Mortgage 600M

In [41]:
executors_per_node = 8
nodes=1
cores_per_executor=1
task_per_core=1

executorcores=1

cache_size=2
total_size=50000
print('executor per node: {:d}\nparallelism: {:d}\nmemory: {:d}m\noffheap:{:d}m'.format(executors_per_node,nodes*executors_per_node*cores_per_executor*task_per_core,int(math.floor(nodes*total_size/(nodes*executors_per_node)))-1024-int(math.floor(cache_size*1024/(nodes*executors_per_node))),int(math.floor(cache_size*1024/(nodes*executors_per_node)))))

label="delinquency_12"


for r in range(0,2):
    params = {'labelCol': "delinquency_12",  #"label"
              'numRound': 100,
              'maxDepth': 8,
              'maxLeaves': 256,
              'alpha': 0.9,
              'eta': 0.1,
              'gamma': 0.1,
              'subsample': 1.0,
              'lambda_': 1.0,
              'scalePosWeight': 2.0,
              'minChildWeight': 30.0,
              'treeMethod': 'gpu_hist',
              'objective': 'reg:squarederror', #if xgboost v0.82 needs to use 'reg:linear'. If >= 0.9, uses squarederror
              'growPolicy': 'depthwise',  
              'numWorkers': executors_per_node*nodes,
              'nthread':cores_per_executor,
              'singlePrecisionHistogram':True,
              'verbosity':2
    }
#    dropcache(clients)
    sc, spark=create_cntx()
    appid = sc.applicationId
    try:
        !echo "" > {home}/notebook/nohup.out
        
        startmonitor(clients, appid, collect_emon=True, collect_nv_smi=True)
        t1 = timeit.default_timer()
        label, features,df = load_parquet(label,file=f"{home}/data/xgboost_36_files_float.dataframe.parquet.8x96")
        params['labelCol']=label
        params['featuresCol']=features
        model, train_time  = run_train(df, params)
        t2 = timeit.default_timer()
    
        get_training_time_from_driver_oldway()
        get_training_time_from_driver()
        stopmonitor(clients, sc, appid)
        display(HTML(('End 2 end. Time(sec): <font size=6pt color=red>{:f}</font>'.format(t2-t1))))
    finally:
        killsar(clients)
        sc.stop()

executor per node: 8
parallelism: 8
memory: 4970m
offheap:256m
ip-172-31-29-158 : 8   83877	83879	83863	83878	83881	83864	83880	83869
app-20210217050842-0008
Wed Feb 17 05:08:44 UTC 2021
root@ip-172-31-29-158: Permission denied (publickey).
Start time : 6868.358113223


,0,1
0,alpha,0.9
1,min_child_weight,30.0
2,sample_type,uniform
3,base_score,0.5
4,rabit_timeout,-1
5,colsample_bylevel,1.0
6,grow_policy,depthwise
7,skip_drop,0.0
8,lambda_bias,0.0
9,silent,0


,0,1
0,alpha,0.9
1,min_child_weight,30.0
2,sample_type,uniform
3,base_score,0.5
4,rabit_timeout,-1
5,colsample_bylevel,1.0
6,grow_policy,depthwise
7,skip_drop,0.0
8,lambda_bias,0.0
9,silent,0


/home/ubuntu
bash: emon: command not found
http://10.0.0.34:18080/history/app-20210217050842-0008/jobs
http://sr525:1088/tracing_examples/trace_viewer.html#/tracing/test_data/app-20210217050842-0008.json
app-20210217050842-0008


ip-172-31-29-158 : 8   88497	88512	88498	88516	88503	88493	88495	88494
app-20210217051421-0009
Wed Feb 17 05:14:23 UTC 2021
root@ip-172-31-29-158: Permission denied (publickey).
Start time : 7193.703939253


,0,1
0,alpha,0.9
1,min_child_weight,30.0
2,sample_type,uniform
3,base_score,0.5
4,rabit_timeout,-1
5,colsample_bylevel,1.0
6,grow_policy,depthwise
7,skip_drop,0.0
8,lambda_bias,0.0
9,silent,0


,0,1
0,alpha,0.9
1,min_child_weight,30.0
2,sample_type,uniform
3,base_score,0.5
4,rabit_timeout,-1
5,colsample_bylevel,1.0
6,grow_policy,depthwise
7,skip_drop,0.0
8,lambda_bias,0.0
9,silent,0


/home/ubuntu
bash: emon: command not found
http://10.0.0.34:18080/history/app-20210217051421-0009/jobs
http://sr525:1088/tracing_examples/trace_viewer.html#/tracing/test_data/app-20210217051421-0009.json
app-20210217051421-0009


In [47]:
executors_per_node = 8
nodes=1
cores_per_executor=1
task_per_core=1

executorcores=1

cache_size=2
total_size=50000
print('executor per node: {:d}\nparallelism: {:d}\nmemory: {:d}m\noffheap:{:d}m'.format(executors_per_node,nodes*executors_per_node*cores_per_executor*task_per_core,int(math.floor(nodes*total_size/(nodes*executors_per_node)))-1024-int(math.floor(cache_size*1024/(nodes*executors_per_node))),int(math.floor(cache_size*1024/(nodes*executors_per_node)))))
label="label"
sc, spark=create_cntx()


executor per node: 8
parallelism: 8
memory: 4970m
offheap:256m
ip-172-31-29-158 : 8   8032	8025	8026	8027	8028	8029	8030	8031
app-20210217053940-0013


In [48]:
df=spark.read.parquet(f"{home}/data/xgboost_36_files_float.dataframe.parquet.8x96")

In [49]:
df=df.repartition(64)
df.write.parquet(f"{home}/data/xgboost_36_files_float.dataframe.parquet.64")

In [51]:
df=df.repartition(8)
df.write.parquet(f"{home}/data/xgboost_36_files_float.dataframe.parquet.8")

In [52]:
sc.stop()

In [53]:
executors_per_node = 8
nodes=1
cores_per_executor=1
task_per_core=1

executorcores=1

cache_size=2
total_size=50000
print('executor per node: {:d}\nparallelism: {:d}\nmemory: {:d}m\noffheap:{:d}m'.format(executors_per_node,nodes*executors_per_node*cores_per_executor*task_per_core,int(math.floor(nodes*total_size/(nodes*executors_per_node)))-1024-int(math.floor(cache_size*1024/(nodes*executors_per_node))),int(math.floor(cache_size*1024/(nodes*executors_per_node)))))

label="delinquency_12"


for r in range(0,2):
    params = {'labelCol': "delinquency_12",  #"label"
              'numRound': 100,
              'maxDepth': 8,
              'maxLeaves': 256,
              'alpha': 0.9,
              'eta': 0.1,
              'gamma': 0.1,
              'subsample': 1.0,
              'lambda_': 1.0,
              'scalePosWeight': 2.0,
              'minChildWeight': 30.0,
              'treeMethod': 'gpu_hist',
              'objective': 'reg:squarederror', #if xgboost v0.82 needs to use 'reg:linear'. If >= 0.9, uses squarederror
              'growPolicy': 'depthwise',  
              'numWorkers': executors_per_node*nodes,
              'nthread':cores_per_executor,
              'singlePrecisionHistogram':True,
              'verbosity':2
    }
#    dropcache(clients)
    sc, spark=create_cntx()
    appid = sc.applicationId
    try:
        !echo "" > {home}/notebook/nohup.out
        
        startmonitor(clients, appid, collect_emon=True, collect_nv_smi=True)
        t1 = timeit.default_timer()
        label, features,df = load_parquet(label,file=f"{home}/data/xgboost_36_files_float.dataframe.parquet.8")
        params['labelCol']=label
        params['featuresCol']=features
        model, train_time  = run_train(df, params)
        t2 = timeit.default_timer()
    
        get_training_time_from_driver_oldway()
        get_training_time_from_driver()
        stopmonitor(clients, sc, appid)
        display(HTML(('End 2 end. Time(sec): <font size=6pt color=red>{:f}</font>'.format(t2-t1))))
    finally:
        killsar(clients)
        sc.stop()

executor per node: 8
parallelism: 8
memory: 4970m
offheap:256m
ip-172-31-29-158 : 8   10913	10916	10904	10920	10905	10921	10910	10926
app-20210217054721-0014
Wed Feb 17 05:47:23 UTC 2021
root@ip-172-31-29-158: Permission denied (publickey).
Start time : 9187.323800181


,0,1
0,alpha,0.9
1,min_child_weight,30.0
2,sample_type,uniform
3,base_score,0.5
4,rabit_timeout,-1
5,colsample_bylevel,1.0
6,grow_policy,depthwise
7,skip_drop,0.0
8,lambda_bias,0.0
9,silent,0


,0,1
0,alpha,0.9
1,min_child_weight,30.0
2,sample_type,uniform
3,base_score,0.5
4,rabit_timeout,-1
5,colsample_bylevel,1.0
6,grow_policy,depthwise
7,skip_drop,0.0
8,lambda_bias,0.0
9,silent,0


/home/ubuntu
bash: emon: command not found
http://10.0.0.34:18080/history/app-20210217054721-0014/jobs
http://sr525:1088/tracing_examples/trace_viewer.html#/tracing/test_data/app-20210217054721-0014.json
app-20210217054721-0014


ip-172-31-29-158 : 8   15440	15441	15442	15461	15435	15436	15437	15438
app-20210217055126-0015
Wed Feb 17 05:51:28 UTC 2021
root@ip-172-31-29-158: Permission denied (publickey).
Start time : 9404.225674716


,0,1
0,alpha,0.9
1,min_child_weight,30.0
2,sample_type,uniform
3,base_score,0.5
4,rabit_timeout,-1
5,colsample_bylevel,1.0
6,grow_policy,depthwise
7,skip_drop,0.0
8,lambda_bias,0.0
9,silent,0


,0,1
0,alpha,0.9
1,min_child_weight,30.0
2,sample_type,uniform
3,base_score,0.5
4,rabit_timeout,-1
5,colsample_bylevel,1.0
6,grow_policy,depthwise
7,skip_drop,0.0
8,lambda_bias,0.0
9,silent,0


/home/ubuntu
bash: emon: command not found
http://10.0.0.34:18080/history/app-20210217055126-0015/jobs
http://sr525:1088/tracing_examples/trace_viewer.html#/tracing/test_data/app-20210217055126-0015.json
app-20210217055126-0015


In [ ]:
executors_per_node = 8
nodes=1
cores_per_executor=1
task_per_core=1

executorcores=1

cache_size=2
total_size=50000
print('executor per node: {:d}\nparallelism: {:d}\nmemory: {:d}m\noffheap:{:d}m'.format(executors_per_node,nodes*executors_per_node*cores_per_executor*task_per_core,int(math.floor(nodes*total_size/(nodes*executors_per_node)))-1024-int(math.floor(cache_size*1024/(nodes*executors_per_node))),int(math.floor(cache_size*1024/(nodes*executors_per_node)))))

label="delinquency_12"


for r in range(0,1):
    params = {'labelCol': "delinquency_12",  #"label"
              'numRound': 100,
              'maxDepth': 8,
              'maxLeaves': 256,
              'alpha': 0.9,
              'eta': 0.1,
              'gamma': 0.1,
              'subsample': 1.0,
              'lambda_': 1.0,
              'scalePosWeight': 2.0,
              'minChildWeight': 30.0,
              'treeMethod': 'gpu_hist',
              'objective': 'reg:squarederror', #if xgboost v0.82 needs to use 'reg:linear'. If >= 0.9, uses squarederror
              'growPolicy': 'depthwise',  
              'numWorkers': executors_per_node*nodes,
              'nthread':cores_per_executor,
              'singlePrecisionHistogram':True,
              'verbosity':2
    }
#    dropcache(clients)
    sc, spark=create_cntx()
    appid = sc.applicationId
    try:
        !echo "" > {home}/notebook/nohup.out
        
        startmonitor(clients, appid, collect_emon=True, collect_nv_smi=True)
        t1 = timeit.default_timer()
        label, features,df = load_parquet(label,file=f"{home}/data/xgboost_36_files_float.dataframe.parquet.64")
        params['labelCol']=label
        params['featuresCol']=features
        model, train_time  = run_train(df, params)
        t2 = timeit.default_timer()
    
        get_training_time_from_driver_oldway()
        get_training_time_from_driver()
        stopmonitor(clients, sc, appid)
        display(HTML(('End 2 end. Time(sec): <font size=6pt color=red>{:f}</font>'.format(t2-t1))))
    finally:
        killsar(clients)
        sc.stop()

# Criteo 2 days

In [42]:
executors_per_node = 8
nodes=1
cores_per_executor=1
task_per_core=1

executorcores=1

cache_size=2
total_size=50000
print('executor per node: {:d}\nparallelism: {:d}\nmemory: {:d}m\noffheap:{:d}m'.format(executors_per_node,nodes*executors_per_node*cores_per_executor*task_per_core,int(math.floor(nodes*total_size/(nodes*executors_per_node)))-1024-int(math.floor(cache_size*1024/(nodes*executors_per_node))),int(math.floor(cache_size*1024/(nodes*executors_per_node)))))

label="_c0"



for r in range(0,2):
    params = {
        'maxDepth': 10,
        'numRound': 200,
        'objective': 'binary:logistic',
        'evalMetric': 'logloss',
        'alpha': 3.0,
        'eta': 0.2,
        'gamma': 0.4,
        'lambda_': 100.0,
        'subsample': 0.8,
        'colsampleBytree': 0.8,
        'minChildWeight': 20.0,
        'treeMethod': 'gpu_hist',
        'growPolicy': 'depthwise',
        'singlePrecisionHistogram':True,
        'numWorkers': executors_per_node*nodes,
        'verbosity': 2
    }
    #dropcache(clients)
    !echo "" > {home}/notebook/nohup.out
    sc, spark=create_cntx()
    appid = sc.applicationId
    try:
        
        startmonitor(clients, appid, collect_emon=True, collect_nv_smi=True, collect_pid=True)
        t1 = timeit.default_timer()
        label, features,df = load_parquet(label,file=f"{home}/data/criteo-dataframe-float-parquet-2days")
        params['labelCol']=label
        params['featuresCol']=features
        model, train_time  = run_train_regressor(df, params)
        t2 = timeit.default_timer()
        
        get_training_time_from_driver_oldway()
        get_training_time_from_driver()
        stopmonitor(clients, sc, appid)
        display(HTML(('End 2 end. Time(sec): <font size=6pt color=red>{:f}</font>'.format(t2-t1))))
    finally:
        killsar(clients)
        sc.stop()

executor per node: 8
parallelism: 8
memory: 4970m
offheap:256m
ip-172-31-29-158 : 8   93077	93079	93078	93081	93083	93082	93085	93086
app-20210217051946-0010
Wed Feb 17 05:19:48 UTC 2021
root@ip-172-31-29-158: Permission denied (publickey).
Start time : 7519.368330292


,0,1
0,alpha,3.0
1,min_child_weight,20.0
2,sample_type,uniform
3,base_score,0.5
4,rabit_timeout,-1
5,colsample_bylevel,1.0
6,grow_policy,depthwise
7,skip_drop,0.0
8,lambda_bias,0.0
9,silent,0


,0,1
0,alpha,3.0
1,min_child_weight,20.0
2,sample_type,uniform
3,base_score,0.5
4,rabit_timeout,-1
5,colsample_bylevel,1.0
6,grow_policy,depthwise
7,skip_drop,0.0
8,lambda_bias,0.0
9,silent,0


/home/ubuntu
bash: emon: command not found
http://10.0.0.34:18080/history/app-20210217051946-0010/jobs
http://sr525:1088/tracing_examples/trace_viewer.html#/tracing/test_data/app-20210217051946-0010.json
app-20210217051946-0010


ip-172-31-29-158 : 8   98257	98256	98259	98258	98261	98260	98279	98255
app-20210217052339-0011
Wed Feb 17 05:23:41 UTC 2021
root@ip-172-31-29-158: Permission denied (publickey).
Start time : 7754.333709847


,0,1
0,alpha,3.0
1,min_child_weight,20.0
2,sample_type,uniform
3,base_score,0.5
4,rabit_timeout,-1
5,colsample_bylevel,1.0
6,grow_policy,depthwise
7,skip_drop,0.0
8,lambda_bias,0.0
9,silent,0


,0,1
0,alpha,3.0
1,min_child_weight,20.0
2,sample_type,uniform
3,base_score,0.5
4,rabit_timeout,-1
5,colsample_bylevel,1.0
6,grow_policy,depthwise
7,skip_drop,0.0
8,lambda_bias,0.0
9,silent,0


/home/ubuntu
bash: emon: command not found
http://10.0.0.34:18080/history/app-20210217052339-0011/jobs
http://sr525:1088/tracing_examples/trace_viewer.html#/tracing/test_data/app-20210217052339-0011.json
app-20210217052339-0011


In [54]:
executors_per_node = 8
nodes=1
cores_per_executor=1
task_per_core=1

executorcores=1

cache_size=2
total_size=50000
print('executor per node: {:d}\nparallelism: {:d}\nmemory: {:d}m\noffheap:{:d}m'.format(executors_per_node,nodes*executors_per_node*cores_per_executor*task_per_core,int(math.floor(nodes*total_size/(nodes*executors_per_node)))-1024-int(math.floor(cache_size*1024/(nodes*executors_per_node))),int(math.floor(cache_size*1024/(nodes*executors_per_node)))))
label="label"
sc, spark=create_cntx()


executor per node: 8
parallelism: 8
memory: 4970m
offheap:256m
ip-172-31-29-158 : 8   20016	20017	20021	20024	20010	20027	20013	20014
app-20210217055551-0016


In [55]:
df=spark.read.parquet(f"{home}/data/criteo-dataframe-float-parquet-2days")

In [56]:
df=df.repartition(8)
df.write.parquet(f"{home}/data/criteo-dataframe-float-parquet-2days.8")

In [57]:
sc.stop()

In [58]:
executors_per_node = 8
nodes=1
cores_per_executor=1
task_per_core=1

executorcores=1

cache_size=2
total_size=50000
print('executor per node: {:d}\nparallelism: {:d}\nmemory: {:d}m\noffheap:{:d}m'.format(executors_per_node,nodes*executors_per_node*cores_per_executor*task_per_core,int(math.floor(nodes*total_size/(nodes*executors_per_node)))-1024-int(math.floor(cache_size*1024/(nodes*executors_per_node))),int(math.floor(cache_size*1024/(nodes*executors_per_node)))))

label="_c0"



for r in range(0,1):
    params = {
        'maxDepth': 10,
        'numRound': 200,
        'objective': 'binary:logistic',
        'evalMetric': 'logloss',
        'alpha': 3.0,
        'eta': 0.2,
        'gamma': 0.4,
        'lambda_': 100.0,
        'subsample': 0.8,
        'colsampleBytree': 0.8,
        'minChildWeight': 20.0,
        'treeMethod': 'gpu_hist',
        'growPolicy': 'depthwise',
        'singlePrecisionHistogram':True,
        'numWorkers': executors_per_node*nodes,
        'verbosity': 2
    }
    #dropcache(clients)
    !echo "" > {home}/notebook/nohup.out
    sc, spark=create_cntx()
    appid = sc.applicationId
    try:
        
        startmonitor(clients, appid, collect_emon=True, collect_nv_smi=True, collect_pid=True)
        t1 = timeit.default_timer()
        label, features,df = load_parquet(label,file=f"{home}/data/criteo-dataframe-float-parquet-2days.8")
        params['labelCol']=label
        params['featuresCol']=features
        model, train_time  = run_train_regressor(df, params)
        t2 = timeit.default_timer()
        
        get_training_time_from_driver_oldway()
        get_training_time_from_driver()
        stopmonitor(clients, sc, appid)
        display(HTML(('End 2 end. Time(sec): <font size=6pt color=red>{:f}</font>'.format(t2-t1))))
    finally:
        killsar(clients)
        sc.stop()

executor per node: 8
parallelism: 8
memory: 4970m
offheap:256m
ip-172-31-29-158 : 8   22194	22196	22200	22201	22186	22202	22187	22188
app-20210217055836-0017
Wed Feb 17 05:58:38 UTC 2021
root@ip-172-31-29-158: Permission denied (publickey).
Start time : 9850.044056138


,0,1
0,alpha,3.0
1,min_child_weight,20.0
2,sample_type,uniform
3,base_score,0.5
4,rabit_timeout,-1
5,colsample_bylevel,1.0
6,grow_policy,depthwise
7,skip_drop,0.0
8,lambda_bias,0.0
9,silent,0


,0,1
0,alpha,3.0
1,min_child_weight,20.0
2,sample_type,uniform
3,base_score,0.5
4,rabit_timeout,-1
5,colsample_bylevel,1.0
6,grow_policy,depthwise
7,skip_drop,0.0
8,lambda_bias,0.0
9,silent,0


/home/ubuntu
bash: emon: command not found
http://10.0.0.34:18080/history/app-20210217055836-0017/jobs
http://sr525:1088/tracing_examples/trace_viewer.html#/tracing/test_data/app-20210217055836-0017.json
app-20210217055836-0017


In [80]:
executors_per_node = 8
nodes=1
cores_per_executor=1
task_per_core=1

executorcores=1

cache_size=2
total_size=50000
print('executor per node: {:d}\nparallelism: {:d}\nmemory: {:d}m\noffheap:{:d}m'.format(executors_per_node,nodes*executors_per_node*cores_per_executor*task_per_core,int(math.floor(nodes*total_size/(nodes*executors_per_node)))-1024-int(math.floor(cache_size*1024/(nodes*executors_per_node))),int(math.floor(cache_size*1024/(nodes*executors_per_node)))))

label="_c0"



for r in range(0,1):
    params = {
        'maxDepth': 10,
        'numRound': 200,
        'objective': 'binary:logistic',
        'evalMetric': 'logloss',
        'alpha': 3.0,
        'eta': 0.2,
        'gamma': 0.4,
        'lambda_': 100.0,
        'subsample': 0.8,
        'colsampleBytree': 0.8,
        'minChildWeight': 20.0,
        'treeMethod': 'gpu_hist',
        'growPolicy': 'depthwise',
        'singlePrecisionHistogram':True,
        'numWorkers': executors_per_node*nodes,
        'verbosity': 2
    }
    #dropcache(clients)
    !echo "" > {home}/notebook/nohup.out
    sc, spark=create_cntx()
    appid = sc.applicationId
    try:
        
        startmonitor(clients, appid, collect_emon=True, collect_nv_smi=True, collect_pid=True)
        t1 = timeit.default_timer()
        label, features,df = load_parquet(label,file=f"{home}/data/criteo-dataframe-float-parquet-2days.8")
        params['labelCol']=label
        params['featuresCol']=features
        model, train_time  = run_train_regressor(df, params)
        t2 = timeit.default_timer()
        
        get_training_time_from_driver_oldway()
        get_training_time_from_driver()
        stopmonitor(clients, sc, appid)
        display(HTML(('End 2 end. Time(sec): <font size=6pt color=red>{:f}</font>'.format(t2-t1))))
    finally:
        killsar(clients)
        sc.stop()

executor per node: 8
parallelism: 8
memory: 4970m
offheap:256m
ip-172-31-29-158 : 8   63522	63523	63529	63531	63537	63510	63511	63512
app-20210217071653-0027
Wed Feb 17 07:16:54 UTC 2021
root@ip-172-31-29-158: Permission denied (publickey).
Start time : 14559.295128716


Py4JJavaError: An error occurred while calling o7706.fit.
: ml.dmlc.xgboost4j.java.XGBoostError: XGBoostModel training failed
	at ml.dmlc.xgboost4j.scala.spark.XGBoost$.postTrackerReturnProcessing(XGBoost.scala:749)
	at ml.dmlc.xgboost4j.scala.spark.rapids.GpuXGBoost$.trainDistributedOnGpu(GpuXGBoost.scala:198)
	at ml.dmlc.xgboost4j.scala.spark.rapids.GpuXGBoost$.trainOnGpu(GpuXGBoost.scala:150)
	at ml.dmlc.xgboost4j.scala.spark.rapids.GpuXGBoost$.fitOnGpu(GpuXGBoost.scala:111)
	at ml.dmlc.xgboost4j.scala.spark.XGBoostRegressor.fit(XGBoostRegressor.scala:170)
	at ml.dmlc.xgboost4j.scala.spark.XGBoostRegressor.fit(XGBoostRegressor.scala:43)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244